In [1]:
import sys
import os

# Obtener el directorio actual
current_directory = os.getcwd()

# Subir dos niveles al directorio raíz, donde se encuentra la carpeta 'interpretml'
root_path = os.path.abspath(os.path.join(current_directory, '..', '..', '..'))

sys.path.append(root_path)

In [2]:
import pandas as pd
import numpy as np
import interpret
from interpret import show
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.skbn as skbn

# IRIS Dataset

In [3]:
iris = pd.read_csv('data/iris.csv', header=None)
iris.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

# Create a binary problem
iris['species'] = np.where(iris['species'] == 'Iris-setosa', 1, 0)

# Discretize the data
iris['sepal_length'] = pd.qcut(iris['sepal_length'], 4, labels=False)
iris['sepal_width'] = pd.qcut(iris['sepal_width'], 4, labels=False)
iris['petal_length'] = pd.qcut(iris['petal_length'], 4, labels=False)
iris['petal_width'] = pd.qcut(iris['petal_width'], 4, labels=False)

In [4]:
X = iris.drop('species', axis=1)
y = iris['species']

In [5]:
# Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TAN

In [6]:
from interpret.glassbox._tan import TANClassifier

In [7]:
TAN_model = TANClassifier()

In [8]:
dir(TAN_model.TAN_class)

['DirichletCsv',
 'MarkovBlanket',
 'XYfromCSV',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_binary_predict',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_estimator_type',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_nary_predict',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_validate_data',
 '_validate_params',
 'beta',
 'bn',
 'constraints',
 'discretizationNbBins',
 'discretizationStrategy',
 'discretizationThreshold',
 'discretizer',
 'fit',
 'fromModel',
 'fromTrainedModel',
 'get_metadata_

In [9]:
TAN_model.fit(X_train, y_train)

In [10]:
TAN_model.predict(X_test)

array([0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.])

In [11]:
(y_test == TAN_model.predict(X_test)).sum() / len(y_test)

1.0

In [12]:
gnb.sideBySide(TAN_model._model().bn, gnb.getInference(TAN_model._model().bn,size='5!'))

In [13]:
print(dir(TAN_model._model().bn))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__swig_destroy__', '__weakref__', '_repr_html_', 'add', 'addAMPLITUDE', 'addAND', 'addArc', 'addArcs', 'addCOUNT', 'addEXISTS', 'addFORALL', 'addLogit', 'addMAX', 'addMEDIAN', 'addMIN', 'addNoisyAND', 'addNoisyOR', 'addNoisyORCompound', 'addNoisyORNet', 'addOR', 'addSUM', 'addStructureListener', 'addVariables', 'addWeightedArc', 'adjacencyMatrix', 'ancestors', 'arcs', 'beginTopologyTransformation', 'changePotential', 'changeVariableLabel', 'changeVariableName', 'check', 'children', 'clear', 'completeInstantiation', 'connectedComponents', 'cpt', 'dag', 'descendants', 'dim', 'empty', 'endTopologyTransform

In [14]:
TAN_model._model().bn.cpt("petal_length")

(pyAgrum.Potential<double>@000001DDCC79CB30) 
      ||  petal_length                         |
y     ||0        |1        |2        |3        |
------||---------|---------|---------|---------|
0     || 0.0060  | 0.2831  | 0.3434  | 0.3675  |
1     || 0.8659  | 0.1098  | 0.0122  | 0.0122  |

In [15]:
np.log(TAN_model._model().bn.cpt("petal_length")[0, :] / TAN_model._model().bn.cpt("petal_length")[1, :])

array([-4.96794842,  0.94765448,  3.33778273,  3.40560532])

In [16]:
iris['petal_length'].value_counts()

petal_length
0    44
2    41
3    34
1    32
Name: count, dtype: int64

In [17]:
petal_length = np.array(iris['petal_length'])
# calculate the number of data of each possible value
n_petal_length = np.zeros(4)
for i in range(4):
    n_petal_length[i] = np.sum(petal_length == i)
n_petal_length

array([44., 32., 41., 34.])

In [18]:
TAN_global = TAN_model.explain_global()
show(TAN_global)

['sepal_length & petal_length', 'sepal_width & sepal_length', 'petal_length', 'petal_width & petal_length']
['sepal_length & petal_length', 'sepal_width & sepal_length', 'petal_length', 'petal_width & petal_length']


<!-- http://127.0.0.1:7001/2052450505872/ -->

In [19]:
TAN_global.selector

,Name,Type,# Unique,% Non-zero
0,sepal_length & petal_length,interaction,NaN,NaN
1,sepal_width & sepal_length,interaction,NaN,NaN
2,petal_length,categorical,NaN,NaN
3,petal_width & petal_length,interaction,NaN,NaN


In [20]:
TAN_global._internal_obj['specific']

[{'type': 'interaction',
  'left_names': [0.0, 1.0, 2.0, 3.0],
  'right_names': [0.0, 1.0, 2.0, 3.0],
  'scores': array([[ 1.0140549 ,  0.12675171, -2.91777073, -2.91777073],
         [ 0.638659  , -0.33074156, -0.77010822,  1.42711636],
         [ 1.60943791,  0.51082562, -0.84729786,  0.14310084],
         [ 2.77258872,  2.77258872, -0.06062462, -1.03407377]]),
  'scores_range': (0.0, 3.0)},
 {'type': 'interaction',
  'left_names': [0.0, 1.0, 2.0, 3.0],
  'right_names': [0.0, 1.0, 2.0, 3.0],
  'scores': array([[-2.75419136,  1.11004098,  1.65658469,  1.93102154],
         [-3.01062089, -1.84054963,  0.35667494,  3.57555077],
         [-0.55499684, -0.09237332,  0.17589067,  1.13140211],
         [ 0.31015493, -0.33647224, -0.42744401,  1.09861229]]),
  'scores_range': (0.0, 3.0)},
 {'type': 'univariate',
  'names': [0.0, 1.0, 2.0, 3.0],
  'scores': [4.967948418133606,
   -0.9476544832815491,
   -3.33778272674226,
   -3.405605323081021],
  'scores_range': None,
  'upper_bounds': None,

In [21]:
print(dir(TAN_model._model().bn))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__swig_destroy__', '__weakref__', '_repr_html_', 'add', 'addAMPLITUDE', 'addAND', 'addArc', 'addArcs', 'addCOUNT', 'addEXISTS', 'addFORALL', 'addLogit', 'addMAX', 'addMEDIAN', 'addMIN', 'addNoisyAND', 'addNoisyOR', 'addNoisyORCompound', 'addNoisyORNet', 'addOR', 'addSUM', 'addStructureListener', 'addVariables', 'addWeightedArc', 'adjacencyMatrix', 'ancestors', 'arcs', 'beginTopologyTransformation', 'changePotential', 'changeVariableLabel', 'changeVariableName', 'check', 'children', 'clear', 'completeInstantiation', 'connectedComponents', 'cpt', 'dag', 'descendants', 'dim', 'empty', 'endTopologyTransform

In [22]:
TAN_model._model().bn.names()

{'petal_length', 'petal_width', 'sepal_length', 'sepal_width', 'y'}

In [23]:
TAN_model._model().bn.ids(TAN_model._model().bn.names())

[0, 4, 2, 3, 1]